# Task 3: Database Storage (PostgreSQL)

This notebook covers storing cleaned review data in PostgreSQL database.

## Objectives:
- Connect to PostgreSQL database
- Insert cleaned review data
- Verify data integrity
- Run SQL queries for analysis

## Database Schema:
- **banks** table: Bank information
- **reviews** table: Review data with foreign key to banks


In [ ]:
# Import required libraries
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")


## Step 1: Database Configuration

**Note**: Update the DB_CONFIG with your PostgreSQL credentials before running.


In [ ]:
# Database configuration
DB_CONFIG = {
    'host': 'localhost',
    'database': 'bank_reviews',
    'user': 'postgres',  # Update with your PostgreSQL username
    'password': '',  # Update with your PostgreSQL password
    'port': 5432
}

print("💡 Update DB_CONFIG above with your PostgreSQL credentials")


## Step 2: Database Connection


In [ ]:
# Connect to database
try:
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()
    print("✅ Successfully connected to PostgreSQL database")
    
    # Test connection
    cursor.execute("SELECT version();")
    version = cursor.fetchone()
    print(f"📊 PostgreSQL Version: {version[0]}")
    
except psycopg2.Error as e:
    print(f"❌ Error connecting to database: {e}")
    print("\n💡 Make sure:")
    print("   1. PostgreSQL is installed and running")
    print("   2. Database 'bank_reviews' exists")
    print("   3. DB_CONFIG credentials are correct")
    print("   4. Run: createdb bank_reviews")
    print("   5. Run: psql bank_reviews < task3_database/create_database.sql")
    conn = None
    cursor = None


## Step 3: Load Review Data


In [ ]:
# Load review data (prefer data with themes, fallback to sentiment or cleaned)
data_files = [
    '../data/processed/reviews_with_themes.csv',
    '../data/processed/reviews_with_sentiment.csv',
    '../data/processed/reviews_cleaned.csv'
]

df = None
for file_path in data_files:
    try:
        df = pd.read_csv(file_path)
        print(f"✅ Loaded {len(df)} reviews from {file_path}")
        break
    except FileNotFoundError:
        continue

if df is None:
    print("❌ No data file found. Please run preprocessing scripts first.")
else:
    print(f"\n📊 Data Overview:")
    print(f"   Columns: {list(df.columns)}")
    print(f"   Banks: {df['bank'].unique().tolist()}")


## Step 4: Insert Data into Database

**Note**: The actual insertion is done using `insert_data.py`. This notebook demonstrates the process.


In [ ]:
# Insert banks (if not already inserted)
if conn and cursor and df is not None:
    try:
        banks_data = [
            ('CBE', 'Commercial Bank of Ethiopia Mobile'),
            ('BOA', 'Bank of Abyssinia Mobile'),
            ('Dashen', 'Dashen Bank Mobile')
        ]
        
        for bank_name, app_name in banks_data:
            cursor.execute(
                "INSERT INTO banks (bank_name, app_name) VALUES (%s, %s) ON CONFLICT (bank_name) DO NOTHING",
                (bank_name, app_name)
            )
        conn.commit()
        print("✅ Bank data inserted/verified")
    except Exception as e:
        print(f"⚠️  Error inserting banks: {e}")
        conn.rollback()
else:
    print("💡 To insert data, run: python task3_database/insert_data.py")


## Step 5: Verify Data in Database


In [ ]:
# Verify data in database
if conn and cursor:
    try:
        # Count reviews per bank
        query = '''
            SELECT b.bank_name, COUNT(r.review_id) as review_count
            FROM banks b
            LEFT JOIN reviews r ON b.bank_id = r.bank_id
            GROUP BY b.bank_name
            ORDER BY review_count DESC
        '''
        cursor.execute(query)
        
        print("="*60)
        print("📊 REVIEWS PER BANK IN DATABASE")
        print("="*60)
        for row in cursor.fetchall():
            print(f"   {row[0]}: {row[1]} reviews")
        
        # Total reviews
        cursor.execute("SELECT COUNT(*) FROM reviews")
        total = cursor.fetchone()[0]
        print(f"\n📈 Total Reviews: {total}")
        
    except Exception as e:
        print(f"⚠️  Error querying database: {e}")
        print("💡 Make sure data has been inserted first")
else:
    print("💡 Connect to database first to verify data")


## Step 6: SQL Queries for Analysis


In [ ]:
# Average rating per bank
if conn and cursor:
    try:
        query = '''
            SELECT b.bank_name, 
                   ROUND(AVG(r.rating), 2) as avg_rating,
                   COUNT(r.review_id) as review_count
            FROM banks b
            LEFT JOIN reviews r ON b.bank_id = r.bank_id
            GROUP BY b.bank_name
            ORDER BY avg_rating DESC
        '''
        cursor.execute(query)
        
        print("="*60)
        print("⭐ AVERAGE RATING BY BANK")
        print("="*60)
        results = cursor.fetchall()
        for row in results:
            print(f"   {row[0]}: {row[1]}★ ({row[2]} reviews)")
    except Exception as e:
        print(f"⚠️  Error: {e}")


In [ ]:
# Sentiment distribution (if available)
if conn and cursor:
    try:
        query = '''
            SELECT b.bank_name, 
                   r.sentiment_label,
                   COUNT(*) as count
            FROM banks b
            JOIN reviews r ON b.bank_id = r.bank_id
            WHERE r.sentiment_label IS NOT NULL
            GROUP BY b.bank_name, r.sentiment_label
            ORDER BY b.bank_name, r.sentiment_label
        '''
        cursor.execute(query)
        
        print("="*60)
        print("💭 SENTIMENT DISTRIBUTION BY BANK")
        print("="*60)
        results = cursor.fetchall()
        current_bank = None
        for row in results:
            if current_bank != row[0]:
                if current_bank is not None:
                    print()
                current_bank = row[0]
                print(f"{row[0]}:")
            print(f"   {row[1]}: {row[2]} reviews")
    except Exception as e:
        print(f"⚠️  Sentiment data not available or error: {e}")


## Step 7: Close Connection


In [ ]:
# Close database connection
if conn:
    conn.close()
    print("✅ Database connection closed")


## Task 3 Summary

✅ **Completed Steps:**
1. PostgreSQL database setup
2. Database schema creation (banks and reviews tables)
3. Data insertion from CSV files
4. Data verification queries
5. SQL analysis queries

✅ **KPIs Achieved:**
- Working database connection
- 1,000+ reviews stored in database
- SQL queries for data integrity verification

**Next Step**: Proceed to Task 4 for Insights and Recommendations
